In [2]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [3]:
gaiatable=pd.read_csv('makingtable/cleanedgaiadata.csv')

In [4]:
mylc=get_lc_string('TIC-59129133-43-SPOC-120')
#mylc2=get_lc_string('TIC-59129133-5-FFI-30min')
#mylc3=get_lc_string('EPIC-246676629-13-EVEREST-1800')

In [5]:
#multiplot2(mylc,'TIC-59129133-43-SPOC-120',gaiatable)

In [6]:
#multiplot2(mylc2,'TIC-59129133-5-FFI-30min',gaiatable)

In [7]:
# def subgraph_rp( #made by sarah
#         lc,
#         period=None,
#         epoch_time=None,
#         ax=None,
#         bin_points=1,
#         minimum_phase=-0.5,
#         maximum_phase=0.5,
#         method="mean",
#         normalize=True,
#         **kwargs,
#     ) -> matplotlib.axes.Axes:

#         fig = None
    
#         if hasattr(lc, "time_original"):  # folded light curve
#             time = lc.time_original
#         else:
#             time = lc.time

#         if period == None:
#             period = lc.to_periodogram(maximum_period=5).period_at_max_power
#         if epoch_time == None:
#             epoch_time = lc.time[0]
#         if normalize:
#             lc=lc.normalize()

#         foldedlc = lc.fold(period, epoch_time=epoch_time) 
#         blc = foldedlc.bin(u.Quantity(period/200, u.d))
#         blc.sort("flux")
#         min = blc[0][0].value/foldedlc.period.value
#         max = blc[-1][0].value/foldedlc.period.value

#         if (bin_points == 1) and (method in ["mean", "median"]):
#             bin_func = lambda y, e: (y[0], e[0])
#         elif method == "mean":
#             bin_func = lambda y, e: (np.nanmean(y), np.nansum(e**2) ** 0.5 / len(e))

#         s = np.argsort(time.value)
#         x, y, e = time.value[s], lc.flux[s], lc.flux_err[s]
#         med = np.nanmedian(lc.flux)
#         e /= med
#         y /= med

#         # Here `ph` is the phase of each time point x
#         # cyc is the number of cycles that have occurred at each time point x
#         # since the phase 0 before x[0]
#         n = int(
#             period.value
#             / np.nanmedian(np.diff(x))
#             * (maximum_phase - minimum_phase)
#             / bin_points
#         )
#         if n == 1:
#             bin_points = int(maximum_phase - minimum_phase) / (
#                 2 / int(period.value / np.nanmedian(np.diff(x)))
#             )
#             warnings.warn(
#                 "`bin_points` is too high to plot a phase curve, resetting to {}".format(
#                     bin_points
#                 ),
#                 LightkurveWarning,
#             )
#             n = 2
#         ph = x / period.value % 1
#         cyc = np.asarray((x - x % period.value) / period.value, int)
#         cyc -= np.min(cyc)

#         phase = (epoch_time.value % period.value) / period.value
#         ph = ((x - (phase * period.value)) / period.value) % 1
#         cyc = np.asarray(
#             (x - ((x - phase * period.value) % period.value)) / period.value, int
#         )
#         cyc -= np.min(cyc)
#         ph[ph > 0.5] -= 1

#         ar = np.empty((n, np.max(cyc) + 1))
#         ar[:] = np.nan
#         bs = np.linspace(minimum_phase, maximum_phase, n + 1)
#         cycs = np.arange(0, np.max(cyc) + 2)

#         ph_masks = [(ph > bs[jdx]) & (ph <= bs[jdx + 1]) for jdx in range(n)]
#         qual_mask = np.isfinite(y)
#         for cyc1 in np.unique(cyc):
#             cyc_mask = cyc == cyc1
#             if not np.any(cyc_mask):
#                 continue
#             for jdx, ph_mask in enumerate(ph_masks):
#                 if not np.any(cyc_mask & ph_mask & qual_mask):
#                     ar[jdx, cyc1] = np.nan
#                 else:
#                     ar[jdx, cyc1] = bin_func(
#                         y[cyc_mask & ph_mask], e[cyc_mask & ph_mask]
#                     )[0]

    
#         # If the method is average we need to denormalize the plot
#         if method in ["mean", "median"]:
#             median = np.nanmedian(lc.flux.value)
#             if hasattr(median, "mask"):
#                 median = median.filled(np.nan)
#             ar *= median

#         d = np.max(
#             [
#                 np.abs(np.nanmedian(ar) - np.nanpercentile(ar, 5)),
#                 np.abs(np.nanmedian(ar) - np.nanpercentile(ar, 95)),
#             ]
#         )
#         vmin = kwargs.pop("vmin", np.nanmedian(ar) - d)
#         vmax = kwargs.pop("vmax", np.nanmedian(ar) + d)
#         if method in ["mean", "median"]:
#             cmap = kwargs.pop("cmap", "viridis")

#         with plt.style.context("seaborn-v0_8-paper"):
#             if ax == None:
#                 fig, ax = plt.subplots(figsize=(12, cyc.max() * 0.1))
#             # if ax is None:
#             #     _, ax = plt.subplots(figsize=(12, cyc.max() * 0.1))

#             im = ax.pcolormesh(
#                 bs, cycs, ar.T, vmin=vmin, vmax=vmax, cmap=cmap, **kwargs
#             )
#             #documentation https://matplotlib.org/stable/gallery/axes_grid1/demo_colorbar_with_axes_divider.html
#             divider = make_axes_locatable(ax)
#             cax = divider.append_axes("right", size="3%", pad="1%")
#             if fig is not None: #so ax was none before and not part of a multiplot
#                 cbar = fig.colorbar(im, cax=cax)
#             else:
#                 cbar = ax.figure.colorbar(im, cax=cax)

#             ax.axvline(min, 0, 1, color="red")
#             ax.axvline(max, 0, 1, color="red")
            
#             if method in ["mean", "median"]:
#                 unit = "[Normalized Flux]"
#                 if lc.flux.unit is not None:
#                     if lc.flux.unit != u.dimensionless_unscaled:
#                         unit = "[{}]".format(lc.flux.unit.to_string("latex"))
#                 if bin_points == 1:
#                     cbar.set_label("Normalized Flux")
#                 else:
#                     cbar.set_label("Average Flux in Bin {}".format(unit))

#             ax.set_xlabel("Phase")
#             ax.set_ylabel("Cycle")
#             ax.set_ylim(cyc.max(), 0)
#             p2=round(period.value/2,2)
#             p4=round(period.value/4,2)
#             ax.set_xticks(ticks=[-0.5,-0.25,0,0.25,0.5],labels=[-p2,-p4,0,p4,p2])
#             a = cyc.max() * 0.1 / 12.0
#             b = (cyc.max() - cyc.min()) / (bs.max() - bs.min())
#             #ax.set_aspect(a / b)
            
#         if fig != None:
#             return fig

In [8]:
#subgraph_rp(mylc)

In [25]:
def multiplot2(lc, infostring, gaiatable, period=None, bintime=None):
    # ['11','12','13','14'],
    # ['21','22','23','24'],
    # ['31','32','33','34'],
    # ['41','42','43','44'],
    # ['51','52','53','54'],
    # ['61','62','63','64'],
    # ['71','72','73','74'],
    # ['81','82','83','84'],
    # ['91','92','93','94'],
    # ['101','102','103','104']
    
    fig, axs = plt.subplot_mosaic([
        ['11','11','11','14'],
        ['11','11','11','14'],
        ['11','11','11','14'],
        ['11','11','11','14'],
        ['51','51','53','53'],
        ['61','61','53','53'],
        ['71','71','73','73'],
        ['71','71','73','73'],
        ['91','91','91','91'],
        ['91','91','91','91']
    ], figsize=(12,18), layout="constrained")

    if period==None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    if bintime==None:
        bintime=u.Quantity(period/200,u.s)
    
    subgraph_phaselc(lc, ax=axs['11'], period=period, bintime=bintime)
    
    subgraph_pg(lc, ax = axs['73'], period=period)

    subgraph_2phaselc(lc, ax=axs['51'], period=period, bintime=bintime)
    subgraph_hphaselc(lc, ax=axs['61'], period=period, bintime=bintime)

    subgraph_rp(lc,ax=axs['53'], period=period)

    subgraph_CMD(infostring,gaiatable,ax=axs['71'])

    subgraph_text(lc, infostring,gaiatable,ax=axs['14'])
    
    subgraph_fulllc(lc, ax = axs['91'])

    return fig

In [38]:
fig=multiplot2(mylc,'TIC-59129133-43-SPOC-120',gaiatable)
fig.savefig('2-TIC-59129133-43-SPOC-120.png',bbox_inches='tight')